In [10]:
import pymongo

# Establish a connection to the MongoDB server
client = pymongo.MongoClient("mongodb://127.0.0.1:27017/")

# Specify the database and collection
db = client["Gridinsight"]
collection = db["new_billing_records"]

In [3]:
import pandas as pd


def fetch_user_table(user_id, consumer_id):
    """
    Fetches and consolidates user tables for the given user_id and consumer_id.
    Returns a sorted and deduplicated DataFrame.
    """
    query = {"user.user_id": user_id, "user.consumer_id": int(consumer_id)}
    projection = {"user": 1, "_id": 0}

    cursor = collection.find(query, projection)
    user_data = [pd.DataFrame(doc["user"]) for doc in cursor if "user" in doc]

    if user_data:
        df = pd.concat(user_data).drop_duplicates().reset_index(drop=True)
    else:
        df = pd.DataFrame()

    return df


# Example usage
user_id_to_check = 22
consumer_id_to_check = 410039008200
result = fetch_user_table(user_id_to_check, consumer_id_to_check)
print(result)

    consumer_id  pincode                               name  user_id
0  410039008200   440008  M/S HALDIRAM FOODS INT. PVT. LTD.       22


In [4]:
def fetch_bill_info_and_reading_dates(user_id, consumer_id):
    """
    Fetches and consolidates bill_info tables and reading dates for the given user_id and consumer_id.
    Adds the current reading date to each corresponding record in the bill_info table.
    Returns a sorted and deduplicated DataFrame.
    """
    query = {"user.user_id": user_id, "user.consumer_id": int(consumer_id)}
    projection = {"bill_info": 1, "reading_dates": 1, "_id": 0}

    cursor = collection.find(query, projection)
    bill_info_data = []
    for doc in cursor:
        if "bill_info" in doc and "reading_dates" in doc:
            df = pd.DataFrame(doc["bill_info"])
            current_date = doc["reading_dates"].get("current_date", [None])[0]
            df["current_reading_date"] = current_date
            bill_info_data.append(df)

    if bill_info_data:
        df = pd.concat(bill_info_data).drop_duplicates().reset_index(drop=True)
    else:
        df = pd.DataFrame()

    return df

user_id_to_check = 22
consumer_id_to_check = 410039008200
result = fetch_bill_info_and_reading_dates(user_id_to_check, consumer_id_to_check)
result

,tariff,connected_load_kw,contract_demand_kva,percentage_contract_demand,feeder_voltage_kv,prev_highest_bill_demand_kva,sd_held_rs,current_reading_date
0,102 HT-I A old trf HT-I A,2900,2500,70,33,1933,0,2023-06-30
1,102 HT-I A old trf HT-I A,2900,2500,70,33,1890,0,2023-07-31
2,102 HT-I A old trf HT-I A,2900,2500,70,33,1890,0,2023-08-31


In [5]:
def fetch_billing_details(user_id, consumer_id):
    """
    Fetches the billing details table for a given user_id and consumer_id.
    Returns a sorted and deduplicated DataFrame.
    """
    query = {"user.user_id": user_id, "user.consumer_id": int(consumer_id)}
    projection = {"billing_details": 1, "_id": 0}

    cursor = collection.find(query, projection)
    billing_details_data = []
    for doc in cursor:
        if "billing_details" in doc:
            df = pd.DataFrame(doc["billing_details"])
            billing_details_data.append(df)

    if billing_details_data:
        df = pd.concat(billing_details_data).drop_duplicates().reset_index(drop=True)
    else:
        df = pd.DataFrame()

    return df

user_id_to_check = 22
consumer_id_to_check = 410039008200
result = fetch_billing_details(user_id_to_check, consumer_id_to_check)
result

,demand_charges,wheeling_charge_rate,wheeling_charge,energy_charges,tod_tariff_ec,fac_rate,electricity_duty,bulk_consumption_rebate,tax_on_sale_rate,incremental_consumption_rebate,...,bill_as_per_merc,total_current_bill,current_interest,principle_arrears,interest_arrears,total_bill_rounded_rs,delayed_payment_charges,fac,tax_on_sale,pf_penal_charges
0,873250,0.60,371053.80,5021594.76,-85249.70,0.00,0.00,-84191.90,18.00,-25813.50,...,0,6181848.36,0.00,2.24,0.00,6181850.00,77273.10,0.00,111204.90,0
1,873250,0.60,338567.40,4581945.48,-67488.50,0.00,0.00,-75398.91,18.00,0.00,...,0,5752344.17,0.00,-2.30,0.00,5752340.00,71904.30,0.00,101468.70,0
2,899198,0.60,415704.60,5625868.92,-76021.90,0.00,0.00,-96277.38,18.00,-81188.25,...,0,6811745.89,0.00,-1.93,0.00,6811740.00,85146.83,0.00,124461.90,0


In [6]:
def fetch_bill_history_table(user_id, consumer_id):
    """
    Fetches the bill history table for a given user_id and consumer_id.
    Returns the bill history table as a DataFrame if found, otherwise returns an empty DataFrame.
    """
    # Query to retrieve the bill history for the specified user_id and consumer_id
    query = {
        "user.user_id": user_id,
        "user.consumer_id": int(consumer_id)  # Ensure consumer_id is an integer
    }

    # Projection to get only the bill history table
    projection = {
        "bill_history_table": 1,
        "_id": 0  # Exclude the _id field
    }

    # Execute the query and retrieve the documents matching the query
    cursor = collection.find(query, projection)

    # List to store DataFrames
    bill_history_data = []

    # Process each document in the cursor
    for document in cursor:
        if "bill_history_table" in document:
            bill_history_data.append(pd.DataFrame(document["bill_history_table"]))

    # Concatenate all DataFrames and drop duplicates, if any
    if bill_history_data:
        df = pd.concat(bill_history_data).drop_duplicates().reset_index(drop=True)
        df['bill_month'] = pd.to_datetime(df['bill_month'])
        df = df.sort_values(by='bill_month').reset_index(drop=True)
    else:
        df = pd.DataFrame()

    return df


user_id_to_check = 22
consumer_id_to_check = 410039008200
result = fetch_bill_history_table(user_id_to_check, consumer_id_to_check)
result

,bill_month,units_bh,bill_demand_kva,bill_amount
0,2022-06-30,549003,1874,6006375
1,2022-07-31,672776,1933,6766182
2,2022-08-31,763842,1855,7504947
3,2022-09-30,741528,1890,7340860
4,2022-10-31,471846,1730,4897346
5,2022-11-30,489174,1625,4982500
6,2022-12-31,707645,1674,6890497
7,2023-01-31,618656,1625,6092238
8,2023-02-28,603544,1625,5956199
9,2023-03-31,616082,1625,6074200


In [7]:
def fetch_rate_charge_table(user_id, consumer_id):
    """
    Fetches the rate charge table for a given user_id and consumer_id.
    Returns the rate charge table as a DataFrame if found, otherwise returns an empty DataFrame.
    """
    # Query to retrieve the rate charge for the specified user_id and consumer_id
    query = {
        "user.user_id": user_id,
        "user.consumer_id": int(consumer_id)  # Ensure consumer_id is an integer
    }

    # Projection to get only the rate charge table
    projection = {
        "rate_charge": 1,
        "_id": 0  # Exclude the _id field
    }

    # Execute the query and retrieve all documents matching the query
    cursor = collection.find(query, projection)

    # Initialize empty lists to store data
    consumption_types = []
    units_rc = []
    rate_rc = []
    charges_rs_rc = []

    # Iterate over the cursor and extract data
    for document in cursor:
        if "rate_charge" in document:
            rate_charge_data = document["rate_charge"]
            consumption_types.extend(rate_charge_data["consumption_type"])
            units_rc.extend(rate_charge_data["units_rc"])
            rate_rc.extend(rate_charge_data["rate_rc"])
            charges_rs_rc.extend(rate_charge_data["charges_rs_rc"])

    # Create DataFrame from lists
    df = pd.DataFrame({
        "consumption_type": consumption_types,
        "units_rc": units_rc,
        "rate_rc": rate_rc,
        "charges_rs_rc": charges_rs_rc
    })
    return df

user_id_to_check = 22
consumer_id_to_check = 410039008200
result = fetch_rate_charge_table(user_id_to_check, consumer_id_to_check)
result

,consumption_type,units_rc,rate_rc,charges_rs_rc
0,Industrial,618423,8.12,5021594.76
1,Residential,0,6.95,0.00
2,Commercial,0,12.83,0.00
3,Industrial,564279,8.12,4581945.48
4,Residential,0,6.95,0.00
5,Commercial,0,12.83,0.00
6,Industrial,692841,8.12,5625868.92
7,Residential,0,6.95,0.00
8,Commercial,0,12.83,0.00


In [8]:
def fetch_tod_table(user_id, consumer_id):
    """
    Fetches the TOD (Time of Day) table for a given user_id and consumer_id.
    Returns the TOD table as a DataFrame if found, otherwise returns an empty DataFrame.
    """
    query = {"user.user_id": user_id, "user.consumer_id": int(consumer_id)}
    projection = {"tod": 1, "_id": 0}

    cursor = collection.find(query, projection)

    # Initialize empty lists to store data
    tod_data = []

    # Iterate over the cursor and extract data
    for document in cursor:
        if "tod" in document:
            tod_data.append(document["tod"])

    # Convert the list of dictionaries to a DataFrame
    if tod_data:
        df = pd.concat([pd.DataFrame(d) for d in tod_data], ignore_index=True).drop_duplicates().reset_index(drop=True)
    else:
        df = pd.DataFrame()
    return df

user_id_to_check = 22
consumer_id_to_check = 410039008200
result = fetch_tod_table(user_id_to_check, consumer_id_to_check)
result

,tod_zone,rate_tod,units_tod,demand_tod,charges_rs_tod
0,00:00 Hrs-06:00 Hrs,-1.5,173649,1024,-260473.5
1,06:00Hrs-09:00Hrs & 12:00Hrs-18:00Hrs,0.0,259940,1602,0.0
2,09:00 Hrs-12:00 Hrs,0.8,93649,1565,74919.2
3,18:00 Hrs-22:00 Hrs,1.1,91186,1135,100304.6
4,00:00 Hrs-06:00 Hrs,-1.5,153378,953,-230067.0
5,06:00Hrs-09:00Hrs & 12:00Hrs-18:00Hrs,0.0,238303,1492,0.0
6,09:00 Hrs-12:00 Hrs,0.8,90931,1387,72744.8
7,18:00 Hrs-22:00 Hrs,1.1,81667,982,89833.7
8,00:00 Hrs-06:00 Hrs,-1.5,178978,1178,-268467.0
9,06:00Hrs-09:00Hrs & 12:00Hrs-18:00Hrs,0.8,309469,1802,0.0


In [12]:
def fetch_demand_charge_table(user_id, consumer_id):
    """
    Fetches the demand charge table for a given user_id and consumer_id.
    Returns the demand charge table as a DataFrame if found, otherwise returns an empty DataFrame.
    """
    query = {"user.user_id": user_id, "user.consumer_id": int(consumer_id)}
    projection = {"demand_charge": 1, "_id": 0}

    cursor = collection.find(query, projection)

    # Initialize an empty list to store demand charge data
    demand_charge_data = []

    # Iterate over the cursor and extract data
    for document in cursor:
        if "demand_charge" in document:
            demand_charge_data.append(document["demand_charge"])

    # Convert the list of dictionaries to a DataFrame
    if demand_charge_data:
        df = pd.concat([pd.DataFrame(d) for d in demand_charge_data], ignore_index=True).reset_index(drop=True)
    else:
        df = pd.DataFrame()
    return df

user_id_to_check = 22
consumer_id_to_check = 410039008200
result = fetch_demand_charge_table(user_id_to_check, consumer_id_to_check)
result

,bill_demand_kva,rate_dc,average_pf
0,1750,499.00,0.999
1,1750,499.00,0.999
2,1802,499.00,0.998


In [16]:
def fetch_current_consumption_table(user_id, consumer_id):
    """
    Fetches the current consumption table for a given user_id and consumer_id.
    Returns the current consumption table as a DataFrame if found, otherwise returns an empty DataFrame.
    """
    query = {"user.user_id": user_id, "user.consumer_id": int(consumer_id)}
    projection = {"current_consumption_details": 1, "_id": 0}

    cursor = collection.find(query, projection)

    # Initialize an empty list to store current consumption data
    current_consumption_data = []

    # Iterate over the cursor and extract data
    for document in cursor:
        if "current_consumption_details" in document:
            current_consumption_data.append(document["current_consumption_details"])

    # Convert the list of dictionaries to a DataFrame
    if current_consumption_data:
        df = pd.concat([pd.DataFrame(d) for d in current_consumption_data], ignore_index=True).reset_index(drop=True)
    else:
        df = pd.DataFrame()

    return df

user_id_to_check = 22
consumer_id_to_check = 410039008200
result = fetch_current_consumption_table(user_id_to_check, consumer_id_to_check)
result

,current,previous,difference,multiplying_factor
0,483026.0,462432.5,20593.5,30
1,25329.0,24819.0,510.0,30
2,501816.5,483026.0,18790.5,30
3,25890.5,25329.0,561.5,30
4,524885.0,501816.5,23068.5,30
5,27099.0,25890.5,1208.5,30


In [17]:
def fetch_solar_net_meter_consumption(user_id, consumer_id):
    """
    Fetches the solar net meter consumption table for a given user_id and consumer_id.
    Returns the solar net meter consumption table as a DataFrame if found, otherwise returns an empty DataFrame.
    """
    query = {"user.user_id": user_id, "user.consumer_id": int(consumer_id)}
    projection = {"solar_net_meter_consumption": 1, "_id": 0}

    cursor = collection.find(query, projection)

    # Initialize an empty list to store solar net meter consumption data
    solar_net_meter_consumption_data = []

    # Iterate over the cursor and extract data
    for document in cursor:
        if "solar_net_meter_consumption" in document:
            solar_net_meter_consumption_data.append(document["solar_net_meter_consumption"])

    # Convert the list of dictionaries to a DataFrame
    if solar_net_meter_consumption_data:
        df = pd.concat([pd.DataFrame(d) for d in solar_net_meter_consumption_data], ignore_index=True).drop_duplicates().reset_index(drop=True)
    else:
        df = pd.DataFrame()

    return df

user_id_to_check = 22
consumer_id_to_check = 410039008200
result = fetch_solar_net_meter_consumption(user_id_to_check, consumer_id_to_check)
result

,generation_unit,export,import,adjusted,bank,banking_charges,solar_capacity
0,71092,0,617805,0,0,0,670
1,53656,0,563715,0,0,0,670
2,61494,600,692055,600,0,0,670


In [18]:
print(type(collection))

<class 'pymongo.collection.Collection'>


In [ ]:
([{'average_cost_of_power': 8.584089466271468, 'consumption': 618423.0, 'contract_demand': 2500.0, 'date': '2023-06-30', 'prev_highest_bill_demand': 1933.0, 'sd_amount': 0.0, 'demand_charge': 873250.0, 'period_of_bill': 30, 'avg_consumption_monthly': 620536.5833333334, 'average_cost_of_fixed_charge': 1.4072498277364522}, {'average_cost_of_power': 8.646598881049977, 'consumption': 564279.0, 'contract_demand': 2500.0, 'date': '2023-07-31', 'prev_highest_bill_demand': 1890.0, 'sd_amount': 0.0, 'demand_charge': 873250.0, 'period_of_bill': 31, 'avg_consumption_monthly': 626321.5833333334, 'average_cost_of_fixed_charge': 1.3942518080767614}, {'average_cost_of_power': 8.533773102342384, 'consumption': 692841.0, 'contract_demand': 2500.0, 'date': '2023-08-31', 'prev_highest_bill_demand': 1890.0, 'sd_amount': 0.0, 'demand_charge': 899198.0, 'period_of_bill': 31, 'avg_consumption_monthly': 617280.1666666666, 'average_cost_of_fixed_charge': 1.4567096896303975}], 'M/S HALDIRAM FOODS INT. PVT. LTD.', 410039008200, '102 HT-I A old trf HT-I A', 440008)

([{'average_cost_of_power': 8.584089466271468, 'consumption': 618423.0, 'contract_demand': 2500.0, 'date': '2023-06-30', 'prev_highest_bill_demand': 1933.0, 'sd_amount': 0.0, 'demand_charge': 873250.0, 'period_of_bill': 30, 'avg_consumption_monthly': 620536.5833333334, 'average_cost_of_fixed_charge': 1.4072498277364522}, {'average_cost_of_power': 8.57322369534229, 'consumption': 862870.0, 'contract_demand': 2500.0, 'date': '2023-10-31', 'prev_highest_bill_demand': 1912.0, 'sd_amount': 0.0, 'demand_charge': 902691.0, 'period_of_bill': 31, 'avg_consumption_monthly': 612710.6666666666, 'average_cost_of_fixed_charge': 1.4732744982406052}], 'M/S HALDIRAM FOODS INT. PVT. LTD.', '410039008200', '102 HT-I A old trf HT-I A', 440008)

In [ ]:
([{'year': '2023', 'consumptionData': [{'name': 'Jan', 'value': 618656}, {'name': 'Feb', 'value': 603544}, {'name': 'Mar', 'value': 616082}, {'name': 'Apr', 'value': 685631}, {'name': 'May', 'value': 526712}, {'name': 'Jun', 'value': 618423}, {'name': 'Jul', 'value': 564279}, {'name': 'Aug', 'value': 'N/A'}, {'name': 'Sep', 'value': 'N/A'}, {'name': 'Oct', 'value': 'N/A'}, {'name': 'Nov', 'value': 'N/A'}, {'name': 'Dec', 'value': 'N/A'}]}, {'year': '2022', 'consumptionData': [{'name': 'Jan', 'value': 'N/A'}, {'name': 'Feb', 'value': 'N/A'}, {'name': 'Mar', 'value': 'N/A'}, {'name': 'Apr', 'value': 'N/A'}, {'name': 'May', 'value': 'N/A'}, {'name': 'Jun', 'value': 549003}, {'name': 'Jul', 'value': 672776}, {'name': 'Aug', 'value': 763842}, {'name': 'Sep', 'value': 741528}, {'name': 'Oct', 'value': 471846}, {'name': 'Nov', 'value': 489174}, {'name': 'Dec', 'value': 707645}]}], 410039008200)


([{'year': '2023', 'consumptionData': [{'name': 'Jan', 'value': 618656}, {'name': 'Feb', 'value': 603544}, {'name': 'Mar', 'value': 616082}, {'name': 'Apr', 'value': 685631}, {'name': 'May', 'value': 526712}, {'name': 'Jun', 'value': 618423}, {'name': 'Jul', 'value': 564279}, {'name': 'Aug', 'value': 692841}, {'name': 'Sep', 'value': 757695}, {'name': 'Oct', 'value': 'N/A'}, {'name': 'Nov', 'value': 'N/A'}, {'name': 'Dec', 'value': 'N/A'}]}, {'year': '2022', 'consumptionData': [{'name': 'Jan', 'value': 'N/A'}, {'name': 'Feb', 'value': 'N/A'}, {'name': 'Mar', 'value': 'N/A'}, {'name': 'Apr', 'value': 'N/A'}, {'name': 'May', 'value': 'N/A'}, {'name': 'Jun', 'value': 549003}, {'name': 'Jul', 'value': 672776}, {'name': 'Aug', 'value': 763842}, {'name': 'Sep', 'value': 741528}, {'name': 'Oct', 'value': 471846}, {'name': 'Nov', 'value': 489174}, {'name': 'Dec', 'value': 707645}]}], '410039008200')

In [ ]:
([{'date': '2023-06-30', 'todConsData': [{'period': '00:00 Hrs-06:00 Hrs', 'value': 173649}, {'period': '06:00Hrs-09:00Hrs & 12:00Hrs-18:00Hrs', 'value': 259940}, {'period': '09:00 Hrs-12:00 Hrs', 'value': 93649}, {'period': '18:00 Hrs-22:00 Hrs', 'value': 91186}]}, {'date': '2023-10-31', 'todConsData': [{'period': '00:00 Hrs-06:00 Hrs', 'value': 244471}, {'period': '06:00Hrs-09:00Hrs & 12:00Hrs-18:00Hrs', 'value': 363807}, {'period': '09:00 Hrs-12:00 Hrs', 'value': 111178}, {'period': '18:00 Hrs-22:00 Hrs', 'value': 143414}]}], '410039008200')


([{'date': '2023-06-30', 'todConsData': [{'period': '00:00 Hrs-06:00 Hrs', 'value': 173649}, {'period': '06:00Hrs-09:00Hrs & 12:00Hrs-18:00Hrs', 'value': 259940}, {'period': '09:00 Hrs-12:00 Hrs', 'value': 93649}, {'period': '18:00 Hrs-22:00 Hrs', 'value': 91186}]}, {'date': '2023-07-31', 'todConsData': [{'period': '00:00 Hrs-06:00 Hrs', 'value': 153378}, {'period': '06:00Hrs-09:00Hrs & 12:00Hrs-18:00Hrs', 'value': 238303}, {'period': '09:00 Hrs-12:00 Hrs', 'value': 90931}, {'period': '18:00 Hrs-22:00 Hrs', 'value': 81667}]}, {'date': '2023-08-31', 'todConsData': [{'period': '00:00 Hrs-06:00 Hrs', 'value': 178978}, {'period': '06:00Hrs-09:00Hrs & 12:00Hrs-18:00Hrs', 'value': 309469}, {'period': '09:00 Hrs-12:00 Hrs', 'value': 107961}, {'period': '18:00 Hrs-22:00 Hrs', 'value': 96433}]}], 410039008200)

In [ ]:
([{'date': '2023-06-30', 'generation_unit': 71092.0, 'import': 617805.0, 'adjusted': 0.0, 'bank': 0.0, 'banking_charges': 0.0, 'solar_capacity': 670.0, 'export': 0.0, 'average_cost_of_power': 8.584089466271468, 'captive_consumed_power': 71092.0, 'period_of_bill': 30, 'actual_bill_saving': 71092.0}, {'date': '2023-07-31', 'generation_unit': 71092.0, 'import': 617805.0, 'adjusted': 0.0, 'bank': 0.0, 'banking_charges': 0.0, 'solar_capacity': 670.0, 'export': 0.0, 'average_cost_of_power': 8.584089466271468, 'captive_consumed_power': 71092.0, 'period_of_bill': 31, 'actual_bill_saving': 71092.0}, {'date': '2023-08-31', 'generation_unit': 71092.0, 'import': 617805.0, 'adjusted': 0.0, 'bank': 0.0, 'banking_charges': 0.0, 'solar_capacity': 670.0, 'export': 0.0, 'average_cost_of_power': 8.584089466271468, 'captive_consumed_power': 71092.0, 'period_of_bill': 31, 'actual_bill_saving': 71092.0}], 410039008200)


([{'date': '2023-06-30', 'generation_unit': 71092.0, 'import': 617805.0, 'adjusted': 0.0, 'bank': 0.0, 'banking_charges': 0.0, 'solar_capacity': 670.0, 'export': 0.0, 'average_cost_of_power': 8.584089466271468, 'captive_consumed_power': 71092.0, 'period_of_bill': 30, 'actual_bill_saving': 71092.0}, {'date': '2023-10-31', 'generation_unit': 71092.0, 'import': 617805.0, 'adjusted': 0.0, 'bank': 0.0, 'banking_charges': 0.0, 'solar_capacity': 670.0, 'export': 0.0, 'average_cost_of_power': 8.584089466271468, 'captive_consumed_power': 71092.0, 'period_of_bill': 31, 'actual_bill_saving': 71092.0}], 410039008200)

In [ ]:
([{'date': '2023-06-30', 'demand_charge': 873250.0, 'wheeling_charges': 371053.8, 'energy_charges': 5021594.76, 'tod_tariff_ec': -85249.7, 'fac': 0.0, 'electricity_duty': 0.0, 'bulk_consumption_rebate': -84191.9, 'tax_on_sale': 111204.9, 'incremental_consumption_rebate': -25813.5, 'charges_for_excess_demand': 0.0, 'tax_collection_at_source': 0.0, 'pf_penal_charges': 0.0, 'debit_bill_adjustment': 0.0, 'generation_unit': '71092', 'import': '617805', 'adjusted': '0', 'bank': '0', 'banking_charges': '0', 'solar_capacity': '670', 'export': '0'}, {'date': '2023-07-31', 'demand_charge': 873250.0, 'wheeling_charges': 338567.4, 'energy_charges': 4581945.48, 'tod_tariff_ec': -67488.5, 'fac': 0.0, 'electricity_duty': 0.0, 'bulk_consumption_rebate': -75398.91, 'tax_on_sale': 101468.7, 'incremental_consumption_rebate': 0.0, 'charges_for_excess_demand': 0.0, 'tax_collection_at_source': 0.0, 'pf_penal_charges': 0.0, 'debit_bill_adjustment': 0.0, 'generation_unit': '53656', 'import': '563715', 'adjusted': '0', 'bank': '0', 'banking_charges': '0', 'solar_capacity': '670', 'export': '0'}, {'date': '2023-08-31', 'demand_charge': 899198.0, 'wheeling_charges': 415704.6, 'energy_charges': 5625868.92, 'tod_tariff_ec': -76021.9, 'fac': 0.0, 'electricity_duty': 0.0, 'bulk_consumption_rebate': -96277.38, 'tax_on_sale': 124461.9, 'incremental_consumption_rebate': -81188.25, 'charges_for_excess_demand': 0.0, 'tax_collection_at_source': 0.0, 'pf_penal_charges': 0.0, 'debit_bill_adjustment': 0.0, 'generation_unit': '61494', 'import': '692055', 'adjusted': '600', 'bank': '0', 'banking_charges': '0', 'solar_capacity': '670', 'export': '600'}], 410039008200)


([{'date': '2023-06-30', 'demand_charge': 873250.0, 'wheeling_charges': 371053.8, 'energy_charges': 5021594.76, 'tod_tariff_ec': -85249.7, 'fac': 0.0, 'electricity_duty': 0.0, 'bulk_consumption_rebate': -84191.9, 'tax_on_sale': 111204.9, 'incremental_consumption_rebate': -25813.5, 'charges_for_excess_demand': 0.0, 'tax_collection_at_source': 0.0, 'pf_penal_charges': 0.0, 'debit_bill_adjustment': 0.0, 'generation_unit': Decimal('71092.000'), 'import': Decimal('617805.000'), 'adjusted': Decimal('0.000'), 'bank': Decimal('0.000'), 'banking_charges': Decimal('0.000'), 'solar_capacity': Decimal('670.000'), 'export': Decimal('0.000')}, {'date': '2023-10-31', 'demand_charge': 902691.0, 'wheeling_charges': 517722.0, 'energy_charges': 7006504.4, 'tod_tariff_ec': -120008.7, 'fac': 172574.0, 'electricity_duty': 0.0, 'bulk_consumption_rebate': -126941.57, 'tax_on_sale': 154229.4, 'incremental_consumption_rebate': -206502.0, 'charges_for_excess_demand': 0.0, 'tax_collection_at_source': 0.0, 'pf_penal_charges': 0.0, 'debit_bill_adjustment': 0.0, 'generation_unit': Decimal('115392.000'), 'import': Decimal('856860.000'), 'adjusted': Decimal('30.000'), 'bank': Decimal('0.000'), 'banking_charges': Decimal('0.000'), 'solar_capacity': Decimal('999.000'), 'export': Decimal('30.000')}], '410039008200')

In [ ]:
([{'date': '2023-06-30', 'actual_generation': 71092.0, 'period': 30, 'expected_generation': 64320.0, 'loss_of_generation': 6772.0, 'reduction_in_bill': 98.8630628807085, 'carbon_offseted': 59717.28}, {'date': '2023-07-31', 'actual_generation': 53656.0, 'period': 31, 'expected_generation': 49848.0, 'loss_of_generation': 3808.0, 'reduction_in_bill': 99.0758525933698, 'carbon_offseted': 45071.04}, {'date': '2023-08-31', 'actual_generation': 61494.0, 'period': 31, 'expected_generation': 58155.99999999999, 'loss_of_generation': 3338.0000000000073, 'reduction_in_bill': 99.10531276393439, 'carbon_offseted': 51654.96}], 410039008200)

([{'date': '2023-06-30', 'actual_generation': 71092.0, 'period': 30, 'expected_generation': 64320.0, 'loss_of_generation': 6772.0, 'reduction_in_bill': 98.8630628807085, 'carbon_offseted': 59717.28}, {'date': '2023-10-31', 'actual_generation': 115392.0, 'period': 31, 'expected_generation': 120779.09999999999, 'loss_of_generation': -5387.099999999991, 'reduction_in_bill': 98.62884203101285, 'carbon_offseted': 96929.28}], '410039008200')

In [ ]:
[{'current_date': '2023-06-30', 'contract_demand': 2500.0, 'connected_load': 2900.0, 'tariff': '102 HT-I A old trf HT-I A', 'additional_capacity': 1830.0, 'required_solar_capacity': 264.64484993132163, 'recommended_solar_capacity': 265.0, 'subsidy_amount': '78000', 'feasible_solar_capacity': 265.0, 'consumption': 618423.0, 'avg_cost_of_power': 8.584089466271468, 'monthly_saving_graph': 610260.0883361712, 'carbon_offset': 59717.28}, {'current_date': '2023-07-31', 'contract_demand': 2500.0, 'connected_load': 2900.0, 'tariff': '102 HT-I A old trf HT-I A', 'additional_capacity': 1830.0, 'required_solar_capacity': 267.11202188857965, 'recommended_solar_capacity': 267.5, 'subsidy_amount': '78000', 'feasible_solar_capacity': 267.5, 'consumption': 564279.0, 'avg_cost_of_power': 8.646598881049977, 'monthly_saving_graph': 463941.9095616176, 'carbon_offset': 45071.04}, {'current_date': '2023-08-31', 'contract_demand': 2500.0, 'connected_load': 2900.0, 'tariff': '102 HT-I A old trf HT-I A', 'additional_capacity': 1830.0, 'required_solar_capacity': 263.2560617064041, 'recommended_solar_capacity': 263.5, 'subsidy_amount': '78000', 'feasible_solar_capacity': 263.5, 'consumption': 692841.0, 'avg_cost_of_power': 8.533773102342384, 'monthly_saving_graph': 524775.8431554425, 'carbon_offset': 51654.96}]


[{'current_date': '2023-06-30', 'contract_demand': 2500.0, 'connected_load': 2900.0, 'tariff': '102 HT-I A old trf HT-I A', 'additional_capacity': 1830.0, 'required_solar_capacity': 335.08072024837116, 'recommended_solar_capacity': 335.5, 'subsidy_amount': '78000', 'feasible_solar_capacity': 335.5, 'consumption': 618423.0, 'avg_cost_of_power': 8.584089466271468, 'monthly_saving_graph': 610260.0883361712, 'carbon_offset': 59717.28}, {'current_date': '2023-10-31', 'contract_demand': 2500.0, 'connected_load': 2900.0, 'tariff': '102 HT-I A old trf HT-I A', 'additional_capacity': 1501.0, 'required_solar_capacity': 330.8548391904258, 'recommended_solar_capacity': 331.0, 'subsidy_amount': '78000', 'feasible_solar_capacity': 331.0, 'consumption': 862870.0, 'avg_cost_of_power': 8.57322369534229, 'monthly_saving_graph': 989281.4286529374, 'carbon_offset': 96929.28}]

In [ ]:
[{'Cumulative Cash Flow': array([-7.95000000e+06, -3.95575213e+06,  1.37478692e+04,  3.95848045e+06,
        7.87842184e+06,  1.17735440e+07,  1.56438144e+07,  1.94891960e+07,
        2.33096472e+07,  2.71051215e+07,  3.08755675e+07,  3.46209289e+07,
        3.83411440e+07,  4.20361462e+07,  4.57058630e+07,  4.93502166e+07,
        5.29691233e+07,  5.65624937e+07,  6.01302322e+07,  6.36722369e+07,
        6.71883997e+07,  7.06786059e+07,  7.41427339e+07,  7.75806554e+07,
        8.09922347e+07,  8.43773290e+07]), 'System Price': 7950000.0, 'Maintenance Cost (NPV)': 2095652.5708300218, 'Net Payments': 10045652.570830021, 'Internal Return Rate (IRR)': 0.4961143531486327, 'Return on Investment (ROI)': 468.95963748451743, 'Levelized Cost of Energy (LCOE)': 0.8862284355602833, 'Payback Period': 2}, {'Cumulative Cash Flow': array([-8.02500000e+06, -3.96270682e+06,  7.44532142e+04,  4.08646111e+06,
        8.07329363e+06,  1.20349232e+07,  1.59713178e+07,  1.98824407e+07,
        2.37682507e+07,  2.76287016e+07,  3.14637422e+07,  3.52733164e+07,
        3.90573629e+07,  4.28158147e+07,  4.65485997e+07,  5.02556400e+07,
        5.39368518e+07,  5.75921455e+07,  6.12214254e+07,  6.48245894e+07,
        6.84015289e+07,  7.19521289e+07,  7.54762674e+07,  7.89738154e+07,
        8.24446369e+07,  8.58885882e+07]), 'System Price': 8025000.0, 'Maintenance Cost (NPV)': 2115422.878102003, 'Net Payments': 10140422.878102003, 'Internal Return Rate (IRR)': 0.4999093614005192, 'Return on Investment (ROI)': 472.5371516099152, 'Levelized Cost of Energy (LCOE)': 0.886228435560283, 'Payback Period': 2}, {'Cumulative Cash Flow': array([-7.90500000e+06, -3.95743661e+06, -3.43606271e+04,  3.86420790e+06,
        7.73824475e+06,  1.15877214e+07,  1.54126050e+07,  1.92128581e+07,
        2.29884386e+07,  2.67392999e+07,  3.04653902e+07,  3.41666530e+07,
        3.78430263e+07,  4.14944433e+07,  4.51208314e+07,  4.87221127e+07,
        5.22982033e+07,  5.58490138e+07,  5.93744486e+07,  6.28744058e+07,
        6.63487774e+07,  6.97974487e+07,  7.32202984e+07,  7.66171982e+07,
        7.99880128e+07,  8.33325996e+07]), 'System Price': 7905000.0, 'Maintenance Cost (NPV)': 2083790.386466833, 'Net Payments': 9988790.386466833, 'Internal Return Rate (IRR)': 0.49305943192805834, 'Return on Investment (ROI)': 466.0799512271306, 'Levelized Cost of Energy (LCOE)': 0.8862284355602833, 'Payback Period': 3}]


[{'Cumulative Cash Flow': array([-1.00650000e+07, -5.00813147e+06,  1.74053212e+04,  5.01158562e+06,
        9.97437935e+06,  1.49057510e+07,  1.98056593e+07,  2.46740576e+07,
        2.95108929e+07,  3.43161066e+07,  3.90896335e+07,  4.38314024e+07,
        4.85413352e+07,  5.32193473e+07,  5.78653473e+07,  6.24792365e+07,
        6.70609090e+07,  7.16102515e+07,  7.61271430e+07,  8.06114546e+07,
        8.50630495e+07,  8.94817822e+07,  9.38674990e+07,  9.82200372e+07,
        1.02539225e+08,  1.06824883e+08]), 'System Price': 10065000.0, 'Maintenance Cost (NPV)': 2653175.2358998954, 'Net Payments': 12718175.235899895, 'Internal Return Rate (IRR)': 0.49611435314863295, 'Return on Investment (ROI)': 468.9596374845173, 'Levelized Cost of Energy (LCOE)': 0.8862284355602833, 'Payback Period': 2}, {'Cumulative Cash Flow': array([-9.93000000e+06, -4.94748925e+06,  4.14267772e+03,  4.92487118e+06,
        9.81466642e+06,  1.46734932e+07,  1.95013107e+07,  2.42980726e+07,
        2.90637266e+07,  3.37982146e+07,  3.85014723e+07,  4.31734290e+07,
        4.78140078e+07,  5.24231249e+07,  5.70006900e+07,  6.15466056e+07,
        6.60607671e+07,  7.05430625e+07,  7.49933724e+07,  7.94115693e+07,
        8.37975181e+07,  8.81510752e+07,  9.24720888e+07,  9.67603982e+07,
        1.01015834e+08,  1.05238218e+08]), 'System Price': 9930000.0, 'Maintenance Cost (NPV)': 2617588.682810329, 'Net Payments': 12547588.682810329, 'Internal Return Rate (IRR)': 0.49545465844640413, 'Return on Investment (ROI)': 468.3377719764143, 'Levelized Cost of Energy (LCOE)': 0.8862284355602827, 'Payback Period': 2}]